In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data", one_hot = True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# What we are going to do:

* Write a few helper functions(init_wgts, init_bias, conv2d)
* variables, placeholders, cost, optimizers
* Train CNN


In [31]:
def init_wgts(shape):
    init_random_dist = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(init_random_dist)

In [32]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape = shape)
    return tf.Variable(init_bias_vals)

In [33]:
def conv2d(x, W):
    """
    x -> [batch, H, W, Channels]
    W -> [filter H, Filter W, Channel IN, Channel OUT]
    Use a stride of 1, keep padding same
    """
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding = 'SAME')

In [34]:
def max_pool_2by2(x):
    """
    x --> [batch, h, w, channels], a 4D Tensor
    """
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                          strides=[1,2,2,1],
                          padding = 'SAME')

In [35]:
def convolutional_layer(input_x, shape):
    W= init_wgts(shape)
    b= init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W)+b)

In [59]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_wgts([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [60]:
#placeholders
x= tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape = [None, 10])

In [61]:
x_image = tf.reshape(x, [-1,28,28,1])

In [62]:
#layer 1
convo_1 = convolutional_layer(x_image, shape = [5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

#layer 2
convo_2= convolutional_layer(convo_1_pooling, shape= [2,2,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

#flatten it out
convo_2_flat = tf.reshape(convo_2_pooling, [-1, 7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1064))

In [63]:
hold_prob = tf.placeholder(tf.float32)

In [65]:
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob = hold_prob)

In [66]:
#Y_pred label
y_pred = normal_full_layer(full_one_dropout, 10)

In [67]:
#loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits= y_pred))

In [68]:
#optimizer
optimizer= tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

In [69]:
#init
init = tf.global_variables_initializer()

In [70]:
steps = 50000

In [ ]:
session = tf.Session()

with session as sess:
    sess.run(init)
    
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(500)
        
        #grab from the next batch
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.5})
        
        #print out message every 100
        if i%100 == 0:
            print("Currently on step{}".format(i))
            print("Accuracy is:")
            #Test the train model
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            print(sess.run(acc, feed_dict = {x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}))
            print('\n')

Currently on step0
Accuracy is:
0.1082


Currently on step100
Accuracy is:
0.8887


Currently on step200
Accuracy is:
0.9299


Currently on step300
Accuracy is:
0.9443


Currently on step400
Accuracy is:
0.9542


Currently on step500
Accuracy is:
0.96


Currently on step600
Accuracy is:
0.9663


Currently on step700
Accuracy is:
0.9688


Currently on step800
Accuracy is:
0.9728


Currently on step900
Accuracy is:
0.9733


Currently on step1000
Accuracy is:
0.9764


Currently on step1100
Accuracy is:
0.9785


Currently on step1200
Accuracy is:
0.9787


Currently on step1300
Accuracy is:
0.9805


Currently on step1400
Accuracy is:
0.9808


Currently on step1500
Accuracy is:
0.9824


Currently on step1600
Accuracy is:
0.9832


Currently on step1700
Accuracy is:
0.9838


Currently on step1800
Accuracy is:
0.9844


Currently on step1900
Accuracy is:
0.9852


Currently on step2000
Accuracy is:
0.986


Currently on step2100
Accuracy is:
0.987


Currently on step2200
Accuracy is:
0.9866


Curr